# Preprocessing
First, let's focus on correctly preprocessing the data. Probably this is the key for a successful classification later on.

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import _pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import time
import copy

In [0]:
path = "D:/GitHub/Machine-Learning/Kaggle/"
# path = "C:/Users/Javi/Documents/GitHub/Machine-Learning/Kaggle/"

zf = zipfile.ZipFile(path+'zipped_TrainData.zip', 'r')
df_train = _pickle.loads(zf.open('TrainData.pkl').read())
zf.close()

zf = zipfile.ZipFile(path+'zipped_TestDataUnlabeled.zip', 'r')
df_test = _pickle.loads(zf.open('TestDataUnlabeled.pkl').read())
zf.close()

As we can see, the data contains different columns:
* **ID**: Just the basic index per row
* **ID_sample**: the ID of the physical sample this is coming from. We should think about how to use different experimental results coming from the same sample. For the moment being, let's just treat them as different samples.
* **Different antibiotics**: data of whether the sample is resistant of not to 9 different antibiotics. Our goal is to predict separately these resistances (one binary classifier for each antibiotic).
* **MZ coordinate**: mass spectrometry is defined over the so-called m/z spectra. Therefore, this constitutes the X axis of the plot.
* **Intensity**: Intensity of the spectra at each point (Y values).

Literature:
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6088204/

Basically, it seems that MALDI TOF MS is routinely used for sample identification. Also, it is used for susceptibility testing (antibiotic resistance classification), mainly using the peaks of the spectra.

### Preprocessing of the data

#### Eliminate NaN values

In [ ]:
def clean_nan_samples(spectrum,targets, c, cat):
# if there are any NaN values, we should remove those samples
    if (targets[cat].isnull().sum() > 0).all(): 
#         print('There are NaN values in',cat)
        merged = pd.concat([spectrum , targets],axis=1,copy=True)
        clean = merged.dropna(subset=[cat])
#         print('Dropped ',len(merged)-len(clean))

        Y = clean.iloc[:,-9+c].to_numpy().reshape(-1,)
        X = clean.iloc[:,:-9]

    else:
        Y = targets.iloc[:,c].to_numpy().reshape(-1,)
        X = spectrum.copy(deep=True)
    return X , Y

#### Spectrum in bins

In [0]:
def spectrum_in_bins(df,m,M,bin_size):
    # Now, let's define the mz ranges, and the label associated to each of them (the mean of the limiting values of each bin)
    range_min = []; range_max = []; range_label = [];
    for mz in range(m,M,bin_size):
        range_min.append(mz)
        range_max.append(mz+bin_size)
        range_label.append(np.mean([range_min[-1],range_max[-1]]).astype(int))


    N = len(df)  # number of samples
    L = len(range_min)  # length of new spectrum (number of bins)
    all_data = np.zeros((N,L))
    for idx in range(N): 
        intensity = df[['intensity']].iloc[idx].values[0]
        mzcoord   = df[['coord_mz']].iloc[idx].values[0]
        idx_data_in_bins = np.zeros((1,L))
        for i,mz in enumerate(range_min):
            intensity_range = intensity[(mzcoord > mz) & (mzcoord < (mz+bin_size))]
            if len(intensity_range) > 0 :
                idx_data_in_bins[0,i] = np.max(intensity_range)
            else: # if those mz coordinates are not in that spectrum
                idx_data_in_bins[0,i] = 0   

        # Normalize the amplitude of the spectrum
        idx_data_in_bins[0,:] = idx_data_in_bins[0,:] / np.max(idx_data_in_bins[0,:])
        
        
        all_data[idx,:] = idx_data_in_bins
    new_df = pd.DataFrame(data=all_data, columns = range_label, index = df.index)
    return new_df

### Data split
Separate data in data and targets, as well as in train, validation and test data.

In [0]:
# Extract data (spectra) and targets of the df_train set
data = df_train.iloc[:,-2:]
targets = df_train.iloc[:,1:-2]

m = 2000; M = 20000; 
bin_size = 50;

# Then, split into a train and test_train set
data_train, data_test_train, targets_train, targets_test_train = train_test_split(data, targets, test_size=0.2, random_state=42) # split the data
print('Training samples: '+str(len(data_train))+' and test_train samples: ' + str(len(data_test_train)) )

# apply the bins to all spectra, so that our feature space becomes the same for all samples (make them regular, all the same)
spectrum_train = spectrum_in_bins(data_train,m,M,bin_size)
spectrum_test_train = spectrum_in_bins(data_test_train,m,M,bin_size)
print('Spectrum regularized!')
# these spectrum_... are our X for training

Training samples: 412 and test_train samples: 104


# Unify spectre function


Now lets unify the spectre of the samples so we get the 112 samples with different approaches on spectre treatment: mean, median, etc.

In [2]:
# Take the data from Google Drive
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Colab Notebooks/Kaggle/Kaggle_data/"

zf = zipfile.ZipFile(path+'zipped_TrainData.zip', 'r')
df_train = _pickle.loads(zf.open('TrainData.pkl').read())
zf.close()

zf = zipfile.ZipFile(path+'zipped_TestDataUnlabeled.zip', 'r')
df_test = _pickle.loads(zf.open('TestDataUnlabeled.pkl').read())
zf.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
print(np.shape(df_test))
print(df_test['ID_sample'].nunique())
print(np.shape(df_train))
print(df_train['ID_sample'].nunique())

(218, 3)
112
(516, 12)
257


In [0]:
def spectrum_in_bins(df,m,M,bin_size):
    # Now, let's define the mz ranges, and the label associated to each of them (the mean of the limiting values of each bin)
    range_min = []; range_max = []; range_label = [];
    for mz in range(m,M,bin_size):
        range_min.append(mz)
        range_max.append(mz+bin_size)
        range_label.append(np.mean([range_min[-1],range_max[-1]]).astype(int))


    N = len(df)  # number of samples
    L = len(range_min)  # length of new spectrum (number of bins)
    all_data = np.zeros((N,L))
    

    for idx in range(N): 
        intensity = df[['intensity']].iloc[idx].values[0]
        mzcoord   = df[['coord_mz']].iloc[idx].values[0]
        idx_data_in_bins = np.zeros((1,L))
        for i,mz in enumerate(range_min):
            intensity_range = intensity[(mzcoord > mz) & (mzcoord < (mz+bin_size))]
            if len(intensity_range) > 0 :
                idx_data_in_bins[0,i] = np.max(intensity_range)
            else: # if those mz coordinates are not in that spectrum
                idx_data_in_bins[0,i] = 0   

        # Normalize the amplitude of the spectrum
        idx_data_in_bins[0,:] = idx_data_in_bins[0,:] / np.max(idx_data_in_bins[0,:])
        
        
        # THIS HAS BEEN CHANGED TO MAKE THE TEST SPECTRUM
        all_data[idx,:] = idx_data_in_bins

    new_df = pd.DataFrame(data=all_data, columns = range_label, index = df.index)
    new_df['ID_sample'] = df.ID_sample
    cols = new_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    new_df = new_df[cols]

    return new_df

In [0]:
# Change the way the spectre is handled: Now is MEAN
# Could be changed to the highest value of both or the lowest, susceptible to changes

def get_unique_spectre(spectre):
  
  # MEAN OF THE SPECTRE
  spectre = spectre.groupby('ID_sample').mean().reset_index()
  return spectre

In [0]:
# Change the way the targets are handled: Now is the MAX value

def get_unique_target(target):

  # MAX value of the target
  target = target.groupby('ID_sample').max().reset_index()
  return target

In [55]:
# Extract data (spectra) and targets of the df_train set
data = df_train.iloc[:,-2:]
targets = df_train.iloc[:,:-2]

m = 2000; M = 20000; 
bin_size = 50;

# apply the bins to all spectra, so that our feature space becomes the same for all samples (make them regular, all the same)
spectrum_train = spectrum_in_bins(df_train,m,M,bin_size)
spectrum_test_train = spectrum_in_bins(df_test,m,M,bin_size)
print('Spectrum regularized!')
# these spectrum_... are our X for training

Spectrum regularized!


In [0]:
new_test_spectrum = get_unique_spectre(spectrum_train)
new_target = get_unique_target(targets)

In [0]:
# TESTEO

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1e6, solver='lbfgs')
params = {'C':10.**np.arange(-5,5)}
lr_best_clfs, _, _, lr_AUC_train, lr_AUC_test_train = try_clf_csv(clf,params)

## This is already finished in Jagger_4_